# 多输入工具

本笔记展示了如何使用使用代理需要多个输入的工具。推荐的方法是使用“StructuredTool”类。



In [1]:
import os
os.environ["LANGCHAIN_TRACING"] = "true"

In [2]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0)

In [3]:
from langchain.tools import StructuredTool

def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)

In [4]:
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
agent_executor = initialize_agent([tool], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [5]:
agent_executor.run("What is 3 times 4")



> Entering new AgentExecutor chain...

Thought: I need to multiply 3 and 4
Action:
```
{
  "action": "multiplier",
  "action_input": {"a": 3, "b": 4}
}
```

Observation: 12
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "3 times 4 is 12"
}
```

> Finished chain.


'3 times 4 is 12'

## 使用字符串格式的多输入工具

另一种使用结构化工具之外的方法是使用常规的“Tool”类并接受单个字符串。然后，工具必须处理解析逻辑以从文本中提取相关值，这会将工具表示紧密地耦合到代理提示中。如果基础语言模型无法 reliably 生成结构化 schema，则此方法仍然很有用。

让我们以乘法函数为例。为了使用它，我们将告诉代理将“操作输入”生成为长度为二的逗号分隔列表。然后，我们将编写一个薄包装器，将字符串分割成两个逗号周围的部分，并将两个分析后的端作为整数传递给乘法函数。

In [6]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

以下是乘法函数及其解析字符串作为输入的包装器。

In [7]:
def multiplier(a, b):
    return a * b

def parsing_multiplier(string):
    a, b = string.split(",")
    return multiplier(int(a), int(b))

In [8]:
llm = OpenAI(temperature=0)
tools = [
    Tool(
        name = "Multiplier",
        func=parsing_multiplier,
        description="useful for when you need to multiply two numbers together. The input to this tool should be a comma separated list of numbers of length two, representing the two numbers you want to multiply together. For example, `1,2` would be the input if you wanted to multiply 1 by 2."
    )
]
mrkl = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [9]:
mrkl.run("What is 3 times 4")



> Entering new AgentExecutor chain...
 I need to multiply two numbers
Action: Multiplier
Action Input: 3,4
Observation: 12
Thought: I now know the final answer
Final Answer: 3 times 4 is 12

> Finished chain.


'3 times 4 is 12'